In [1]:
import faiss, numpy as np

# 1. 生成/加载你的向量
xb = np.random.random((100_000, 1024)).astype(np.float32)   # 纯 ndarray

# 2. 如果 xb 来自 pandas / torch，一定加这两句保险
xb = np.asarray(xb, dtype=np.float32)
xb = np.ascontiguousarray(xb)

# 3. 建 IVF 索引
d = xb.shape[1]
nlist = 4096
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

index.train(xb)      # 现在不会报 "input not a numpy array"
index.add(xb)

In [5]:
# 1. 造 3 条查询向量（同维度 768）
xq = np.random.random((1, 1024)).astype(np.float32)

# 2. 指定要扫多少个倒排桶（越大越准越慢）
index.nprobe = 64          # 经验初值：nlist//60 ~ nlist//100

# 3. 执行搜索：返回距离和编号
k = 1                     # 取 top-5
D, I = index.search(xq, k) # D=(3,5)距离  I=(3,5)编号

print("最近邻编号:\n", I)


最近邻编号:
 [[93937]]


In [7]:
from sentence_transformers import SentenceTransformer

d:\code\competiton\饿了么智慧点餐\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
model = SentenceTransformer('BAAI/bge-m3')

In [21]:
def querys( querys: list[str], EmbeddingModel, k: int = 1):
    # 查询函数，返回与query最相似的k个文档（稠密查询）
    query_vector = EmbeddingModel.encode(querys)
    D,I = index.search(query_vector, k)
    print(I)
    for q_idx, neighbor_idx in enumerate(I):
        print(f"查询{q_idx} 的 top-5 文档：")
        

In [23]:
querys(querys = ['鱼豆腐汤'],EmbeddingModel = model, k=10)

[[88492 92347 55004 96526 37766 45517 61146 90711 97509 18010]]
查询0 的 top-5 文档：
